#1

In [ ]:
import pandas as pd
import re
import networkx as nx
import numpy as np

def parse_itp_file(file_path):
    data = {}
    current_section = None

    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if line.startswith(';') or not line:
                continue
            elif line.startswith('[') and line.endswith(']'):
                current_section = line[1:-1].strip()
                data[current_section] = []
                if current_section in ['angles', 'moleculetype', 'atoms']:
                    next(file)  # Skip one line for certain sections
            else:
                line_data = line.split(';')[0].strip()
                if line_data:
                    if current_section == 'dihedrals':
                        if 'IMPROPER DIHEDRAL' in line_data:
                            next(file)  # Skip the next line for improper dihedrals
                            continue
                        elif 'PROPER DIHEDRAL' in line_data:
                            next(file)  # Skip the next line for proper dihedrals
                            continue
                    data[current_section].append(line_data.split())

    return data

def process_dihedrals_sections(file_path):
    improper_data = []
    proper_data = []
    with open(file_path, 'r') as file:
        section_flag = None  # Tracks the current section
        dihedrals_flag = False  # Flag to indicate dihedrals section
        section_type = None  # Tracks the type of dihedrals section
        for line in file:
            line = line.strip()
            if line.startswith('[ dihedrals ]'):
                dihedrals_flag = True
            elif dihedrals_flag and line.startswith('; IMPROPER DIHEDRAL ANGLES'):
                section_flag = 'improper'
                section_type = 'Improper Dihedrals'
            elif dihedrals_flag and line.startswith('; PROPER DIHEDRAL ANGLES'):
                section_flag = 'proper'
                section_type = 'Proper Dihedrals'
            elif '[ pairs ]' in line:
                break  # Stop collecting data after encountering [ pairs ]
            elif section_flag == 'improper' and line.strip() and not line.startswith(';'):
                improper_data.append(line.split())
            elif section_flag == 'proper' and line.strip() and not line.startswith(';'):
                proper_data.append(line.split())

    # Convert data to dataframes
    df_improper = pd.DataFrame(improper_data, columns=['ai', 'aj', 'ak', 'al', 'funct', 'c0', 'c1', 'c2'])
    df_proper = pd.DataFrame(proper_data, columns=['ai', 'aj', 'ak', 'al', 'funct', 'c0', 'c1', 'c2', 'c3', 'c4', 'c5'])

    return df_improper, df_proper

def parse_gro(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    num_atoms = int(lines[1].strip())

    atom_data = []

    for i in range(2, 2 + num_atoms):
        line = lines[i].strip()

        parts = re.split(r'\s+', line)

        res = parts[0]
        atom_name = parts[1]
        atom_num = int(parts[2])
        x = float(parts[3])
        y = float(parts[4])
        z = float(parts[5])

        atom_data.append([res, atom_name, atom_num, x, y, z])

    df = pd.DataFrame(atom_data, columns=['res', 'atom_name', 'atom_num', 'x', 'y', 'z'])

    return df


# Assuming the files were uploaded
itp_file_path = '7mer.itp'
gro_file_path = '7mer.gro'

parsed_data = parse_itp_file(itp_file_path)
gro_df = parse_gro(gro_file_path)

# Convert the 'atoms' section to a DataFrame
df_atomtypes = pd.DataFrame(parsed_data.get('atomtypes', []), columns=['type', 'type_name', 'mass', 'charge', 'element', 'sigma', 'epsilon'])
df_moleculetype = pd.DataFrame(parsed_data.get('moleculetype', []), columns=['name', 'nrexcl'])
df_atoms = pd.DataFrame(parsed_data.get('atoms', []), columns=['nr', 'type', 'resnr', 'residue', 'atom', 'cgnr', 'charge', 'mass'])
df_bonds = pd.DataFrame(parsed_data.get('bonds', []), columns=['ai', 'aj', 'funct', 'c0', 'c1'])
df_angles = pd.DataFrame(parsed_data.get('angles', []), columns=['ai', 'aj', 'ak', 'funct', 'c0', 'c1'])
df_pairs = pd.DataFrame(parsed_data.get('pairs', []), columns=['ai', 'aj', 'funct'])


In [ ]:
print(df_atomtypes)
print(df_moleculetype)
print(df_atoms)
print(df_bonds)
print(df_angles)
print(df_pairs)

print(gro_df)

        type type_name     mass charge element        sigma      epsilon
0   opls_866      C866  12.0110  0.000       A  3.55000E-01  2.92880E-01
1   opls_833      S833  32.0600  0.000       A  3.60000E-01  1.48532E+00
2   opls_870      H870   1.0080  0.000       A  2.50000E-01  1.25520E-01
3   opls_873      S873  32.0600  0.000       A  3.60000E-01  1.48532E+00
4   opls_828      H828   1.0080  0.000       A  2.50000E-01  1.25520E-01
..       ...       ...      ...    ...     ...          ...          ...
89  opls_880      H880   1.0080  0.000       A  2.42000E-01  1.25520E-01
90  opls_859      H859   1.0080  0.000       A  2.50000E-01  1.25520E-01
91  opls_842      C842  12.0110  0.000       A  3.55000E-01  2.92880E-01
92  opls_892      C892   12.011      0       A     3.50E-01     2.76E-01
93  opls_893      H893    1.008      0       A     2.50E-01     1.26E-01

[94 rows x 7 columns]
  name nrexcl
0  UNK      3
      nr      type resnr residue  atom cgnr   charge     mass
0      1  o

In [ ]:
import pandas as pd

# Define atomic symbols
atomic_symbols = [
    "H", "He", "Li", "Be", "B", "C", "N", "O", "F", "Ne",
    "Na", "Mg", "Al", "Si", "P", "S", "Cl", "Ar", "K", "Ca",
    "Sc", "Ti", "V", "Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Zn",
    "Ga", "Ge", "As", "Se", "Br", "Kr", "Rb", "Sr", "Y", "Zr",
    "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag", "Cd", "In", "Sn",
    "Sb", "Te", "I", "Xe", "Cs", "Ba", "La", "Ce", "Pr", "Nd",
    "Pm", "Sm", "Eu", "Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb",
    "Lu", "Hf", "Ta", "W", "Re", "Os", "Ir", "Pt", "Au", "Hg",
    "Tl", "Pb", "Bi", "Po", "At", "Rn", "Fr", "Ra", "Ac", "Th",
    "Pa", "U", "Np", "Pu", "Am", "Cm", "Bk", "Cf", "Es", "Fm",
    "Md", "No", "Lr", "Rf", "Db", "Sg", "Bh", "Hs", "Mt", "Ds",
    "Rg", "Cn", "Nh", "Fl", "Mc", "Lv", "Ts", "Og"
]

# Define the function
def split_atom_name(df, column_name):
    atom_col = []
    atom_namnum_col = []

    for atom_name in df[column_name]:
        atom = None
        atom_namnum = None

        # Check if the atom name starts with an atomic symbol
        for symbol in atomic_symbols:
            if atom_name.startswith(symbol):
                # Check if there are additional characters after the atomic symbol
                if len(atom_name) > len(symbol):
                    next_char = atom_name[len(symbol)]
                    if next_char.isdigit() or (next_char.isalpha() and next_char.isupper()):
                        atom = symbol
                        atom_namnum = atom_name[len(symbol):]
                        break
                    elif next_char.isalpha() and next_char.islower():
                        # Check if the full atom name matches any atomic symbol
                        if atom_name in atomic_symbols:
                            atom = atom_name
                            break

        if atom is None:
            raise ValueError(f"Atomic symbol not found in atom_name: {atom_name}")

        atom_col.append(atom)
        atom_namnum_col.append(atom_namnum)

    df['atom_nam'] = atom_col
    df['atom_namnum'] = atom_namnum_col
    df.drop(columns=[column_name], inplace=True)

    return df

# Apply the function to gro_df
gro_df = split_atom_name(gro_df, 'atom_name')

# Apply the function to df_atoms
df_atoms = split_atom_name(df_atoms, 'atom')

print(gro_df)
print(df_atoms )

      res  atom_num      x      y      z atom_nam atom_namnum
0    1UNK         1  0.100  0.100  0.000        C         000
1    1UNK         2 -0.038  0.100  0.000        C         001
2    1UNK         3 -0.094  0.100  0.130        C         002
3    1UNK         4  0.006  0.100  0.230        C         003
4    1UNK         5  0.166  0.100  0.158        S         004
..    ...       ...    ...    ...    ...      ...         ...
753  1UNK       754  3.465  0.683 -7.497        H         0KX
754  1UNK       755  3.635  0.796 -7.431        C         0KY
755  1UNK       756  3.692  0.779 -7.520        H         0KZ
756  1UNK       757  3.655  0.895 -7.395        H         0L0
757  1UNK       758  3.664  0.725 -7.356        H         0L1

[758 rows x 7 columns]
      nr      type resnr residue cgnr   charge     mass atom_nam atom_namnum
0      1  opls_800     1     UNK    1  -0.0242  12.0110        C         000
1      2  opls_801     1     UNK    1  -0.1180  12.0110        C         001
2

In [ ]:
print('Atoms section from mol.itp')
print(df_atoms)
print('Bonds section from mol.itp')
print(df_bonds)
print('coordinate information from mol.gro')
print(gro_df)

Atoms section from mol.itp
      nr      type resnr residue cgnr   charge     mass atom_nam atom_namnum
0      1  opls_800     1     UNK    1  -0.0242  12.0110        C         000
1      2  opls_801     1     UNK    1  -0.1180  12.0110        C         001
2      3  opls_804     1     UNK    1   0.0506  32.0600        S         004
3      4  opls_802     1     UNK    1  -0.0301  12.0110        C         002
4      5  opls_805     1     UNK    1   0.1063   1.0080        H         005
..   ...       ...   ...     ...  ...      ...      ...      ...         ...
753  754  opls_893     1     UNK    3   0.0912   1.0080        H         0KX
754  755  opls_892     1     UNK    3  -0.2334  12.0110        C         0KY
755  756  opls_893     1     UNK    3   0.0826   1.0080        H         0KZ
756  757  opls_893     1     UNK    3   0.0826   1.0080        H         0L0
757  758  opls_893     1     UNK    3   0.0826   1.0080        H         0L1

[758 rows x 9 columns]
Bonds section from mol.it

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

# Assuming the dataframes are already loaded as df_bonds, df_atoms, and gro_df

# Ensure ai and aj columns are integers
df_bonds['ai'] = df_bonds['ai'].astype(int)
df_bonds['aj'] = df_bonds['aj'].astype(int)

# Ensure c0 values are floats
df_bonds['c0'] = df_bonds['c0'].astype(float)

# Step 1: Create a dictionary to map atom index to type for df_atoms
atom_type_map = {int(k): v for k, v in df_atoms.set_index('nr')['atom_nam'].to_dict().items()}

# Calculate bond lengths for gro_df
def calculate_distance(row1, row2):
    return np.sqrt((row1['x'] - row2['x'])**2 + (row1['y'] - row2['y'])**2 + (row1['z'] - row2['z'])**2)

# Step 2: Count bond types in df_bonds
df_bonds['bond_type'] = df_bonds.apply(lambda row: (atom_type_map.get(row['ai']), atom_type_map.get(row['aj'])), axis=1)
bond_type_counts = df_bonds['bond_type'].value_counts()

# Step 3: Find matching bonds in gro_df within a 10% tolerance window
def find_matching_bonds(df_bonds, gro_df, tolerance=0.1):
    matching_bonds = []
    for bond_type, count in bond_type_counts.items():
        for i, atom1 in gro_df.iterrows():
            for j, atom2 in gro_df.iterrows():
                if i != j:
                    bond_length = calculate_distance(atom1, atom2)
                    for _, bond_row in df_bonds[df_bonds['bond_type'] == bond_type].iterrows():
                        c0 = float(bond_row['c0'])
                        if abs(bond_length - c0) / c0 <= tolerance:
                            matching_bonds.append({
                                'gro_ai': atom1['atom_num'],
                                'gro_aj': atom2['atom_num'],
                                'bond_type': bond_type,
                                'length': bond_length
                            })
    return matching_bonds

# Find initial matching bonds
initial_matches = find_matching_bonds(df_bonds, gro_df)

# Debugging: Print initial matching bonds
print("Initial Matching Bonds:", initial_matches)

# Step 4: Expand from matched bonds to fit the whole system
def expand_matching_bonds(initial_matches, df_bonds, gro_df, tolerance=0.15):
    final_matches = initial_matches.copy()
    used_atoms = set()
    for match in initial_matches:
        used_atoms.add(match['gro_ai'])
        used_atoms.add(match['gro_aj'])

    # Continue expanding until all bonds are matched
    while len(final_matches) < len(df_bonds):
        new_matches = []
        for match in final_matches:
            for i, atom1 in gro_df.iterrows():
                for j, atom2 in gro_df.iterrows():
                    if i != j and i not in used_atoms and j not in used_atoms:
                        bond_length = calculate_distance(atom1, atom2)
                        for _, bond_row in df_bonds.iterrows():
                            c0 = float(bond_row['c0'])
                            if abs(bond_length - c0) / c0 <= tolerance:
                                new_match = {
                                    'gro_ai': atom1['atom_num'],
                                    'gro_aj': atom2['atom_num'],
                                    'bond_type': bond_row['bond_type'],
                                    'length': bond_length
                                }
                                if new_match not in final_matches and new_match not in new_matches:
                                    new_matches.append(new_match)
                                    used_atoms.add(atom1['atom_num'])
                                    used_atoms.add(atom2['atom_num'])
        final_matches.extend(new_matches)
        if not new_matches:  # Break if no new matches found
            break

    return final_matches

# Expand the initial matches to cover the whole system
final_matches = expand_matching_bonds(initial_matches, df_bonds, gro_df)

# Debugging: Print final matching bonds
print("Final Matching Bonds:", final_matches)

# Step 5: Create and visualize graphs

def create_graph(df_bonds):
    G = nx.Graph()
    for i, row in df_bonds.iterrows():
        G.add_edge(row['ai'], row['aj'])
    return G

def create_graph_from_matches(matches):
    G = nx.Graph()
    for match in matches:
        G.add_edge(match['gro_ai'], match['gro_aj'])
    return G

# Create graphs
G_bonds = create_graph(df_bonds)
G_gro = create_graph_from_matches(final_matches)

# Visualize the graphs
def visualize_graph(G, title):
    pos = nx.spring_layout(G)  # positions for all nodes
    plt.figure(figsize=(10, 7))
    nx.draw(G, pos, with_labels=True, node_size=500, font_size=12)
    plt.title(title)
    plt.show()

visualize_graph(G_bonds, "Bond Structure from df_bonds (itp file)")
visualize_graph(G_gro, "Structure from gro_df")

# Step 6: Perform graph isomorphism

# Define node match function
def node_match(n1, n2):
    return True  # Nodes don't have additional attributes

# Define edge match function
def edge_match(e1, e2):
    return True  # Edges don't have additional attributes

# Check for isomorphism and find mapping
GM = nx.isomorphism.GraphMatcher(G_bonds, G_gro, node_match=node_match, edge_match=edge_match)
if GM.is_isomorphic():
    mapping = GM.mapping
    print("Isomorphism Mapping:", mapping)
else:
    print("Graphs are not isomorphic.")

# Step 7: Reorder atoms in gro_df based on isomorphism mapping

def reorder_atoms(mapping, gro_df):
    inverse_mapping = {v: k for k, v in mapping.items()}
    gro_df['fit'] = gro_df['atom_num'].map(inverse_mapping)
    return gro_df.sort_values(by='fit').reset_index(drop=True)

if GM.is_isomorphic():
    reordered_gro_df = reorder_atoms(GM.mapping, gro_df)
    print(reordered_gro_df)
else:
    print("Cannot reorder atoms because graphs are not isomorphic.")

# Step 8: Rewrite the atom_num column to match the fit column
def update_atom_num(gro_df):
    gro_df['atom_num'] = gro_df['fit']
    return gro_df.drop(columns=['fit'])

# Step 9: Update the atom_nam and atom_namnum columns using a custom hex style
def hex_style_atom_type(index):
    """Generate hex-style atom type based on index."""
    hex_digits = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    base = len(hex_digits)
    result = ''
    index -= 1
    while index > 0:
        index, remainder = divmod(index, base)
        result = hex_digits[remainder] + result
    return result.zfill(3)  # Pad with zeros to ensure three characters

def update_dataframe_with_hex_style(df):
    # Update atom_num to be sequential starting from 1
    df['atom_num'] = range(1, len(df) + 1)

    # Create a custom sequence for atom_namnum using hex_style_atom_type function
    df['atom_namnum'] = [hex_style_atom_type(i) for i in range(1, len(df) + 1)]

    return df

# Step 10: Combine atom_nam and atom_namnum into a new column called atom
def combine_atom_columns(df):
    df['atom_names'] = df['atom_nam'] + df['atom_namnum']
    return df

if GM.is_isomorphic():
    updated_gro_df = update_atom_num(reordered_gro_df)
    updated_gro_df = update_dataframe_with_hex_style(updated_gro_df)
    updated_gro_df = combine_atom_columns(updated_gro_df)
    print(updated_gro_df)
else:
    print("Cannot update atom_num because graphs are not isomorphic.")


KeyboardInterrupt: 

In [ ]:
def write_gro_file(df, gro_file_path, output_file_path):
    # Read the first line from the input.gro file
    with open(gro_file_path, 'r') as file:
        first_line = file.readline().strip()

    # Calculate the total number of atoms
    total_atoms = df['atom_num'].max()

    # Check and convert x, y, z columns to numeric if possible
    numeric_columns = ['x', 'y', 'z']
    for col in numeric_columns:
        try:
            df[col] = pd.to_numeric(df[col])
        except ValueError:
            print(f"Error: Column '{col}' contains non-numeric values.")
            return

    # Prepare the DataFrame content for writing
    df['x'] = df['x'].map('{:.3f}'.format)
    df['y'] = df['y'].map('{:.3f}'.format)
    df['z'] = df['z'].map('{:.3f}'.format)

    # Write to the output.gro file
    with open(output_file_path, 'w') as file:
        file.write(f"{first_line}\n")
        file.write(f"  {total_atoms}\n")

        for _, row in df.iterrows():
            res = row['res']
            atom_names = row['atom_names']
            atom_num = row['atom_num']
            x = row['x']
            y = row['y']
            z = row['z']

            file.write(f"{res:>8}{atom_names:>7}{atom_num:>5}{x:>8}{y:>8}{z:>8}\n")
        file.write(f"   1.00000   1.00000   1.00000\n")

output_file_path = 'output.gro'
write_gro_file(updated_gro_df, gro_file_path, output_file_path)